<a href="https://colab.research.google.com/github/Huhan001/larning/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
link = 'https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip'
zip_in = tf.keras.utils.get_file(origin=link, extract= True, fname='101_food_classes_10_percent.zip')

inspect = os.path.dirname(zip_in)

base = os.path.dirname(zip_in)
getin = os.path.join(base, '101_food_classes_10_percent')

train = os.path.join(getin, 'train')
test = os.path.join(getin, 'test')
!find $test -type d -print

Building flow of image using simple API not imageGenerator

In [ ]:
trainData = tf.keras.preprocessing.image_dataset_from_directory(directory= train,
                                                              batch_size = 64,
                                                              image_size = (150, 150),
                                                              label_mode = 'categorical')
testData = tf.keras.preprocessing.image_dataset_from_directory(directory= test,
                                                              batch_size = 64,
                                                              image_size = (150, 150),
                                                              label_mode = 'categorical')

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
    # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name = 'data_augmentation')

model checkpoint

In [ ]:
# Create checkpoint callback to save model for later use
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True) # only keep the best model weights on file (delete the rest)

Making the model now after processing the data

In [ ]:
base = tf.keras.applications.EfficientNetB0(include_top = False)
base.trainable = False
input = tf.keras.layers.Input(shape = (150,150,3), name ='input_layer')
x = data_augmentation(input)
x = base(x, training=False) # can remain false after data augmentation no need to train
print('shape after base model {}'.format(input))

# 4. If using ResNet50V2, add this to speed up convergence, remove for EfficientNet
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

#6
outputs = tf.keras.layers.Dense(101, activation="softmax", name="output_layer")(x)

#7. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(input, outputs)

#8. Compile the model
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

history = model_0.fit(trainData,
                      epochs= 10,
                      validation_data= testData,
                      validation_steps= len(testData),
                      steps_per_epoch= len(trainData),
                      callbacks=[checkpoint_callback])

shape after base model KerasTensor(type_spec=TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='input_layer'), name='input_layer', description="created by layer 'input_layer'")
After GlobalAveragePooling2D(): (None, 1280)
Epoch 1/10


237/237 [==============================] - 105s 405ms/step - loss: 3.5853 - accuracy: 0.2218 - val_loss: 2.5764 - val_accuracy: 0.4055
Epoch 2/10
237/237 [==============================] - 94s 395ms/step - loss: 2.4877 - accuracy: 0.4231 - val_loss: 2.1828 - val_accuracy: 0.4621
Epoch 3/10
237/237 [==============================] - 93s 391ms/step - loss: 2.1191 - accuracy: 0.4925 - val_loss: 2.0345 - val_accuracy: 0.4849
Epoch 4/10
237/237 [==============================] - 133s 563ms/step - loss: 1.9098 - accuracy: 0.5284 - val_loss: 1.9714 - val_accuracy: 0.4945
Epoch 5/10
237/237 [==============================] - 106s 447ms/step - loss: 1.7589 - accuracy: 0.5637 - val_loss: 1.9290 - val_accuracy: 0.5038
Epoch 6/10
237/237 [==============================] - 94s 397ms/step - loss: 1.6120 - accuracy: 0.5954 - val_loss: 1.9108 - val_accuracy: 0.5056
Epoch 7/10
237/237 [==============================] - 99s 416ms/step - loss: 1.5209 - accuracy: 0.6185 - val_loss: 1.8972 - val_accuracy: 

fine tunning builds upon the model checkpoint and plays with the layers from already trained models.

In [ ]:
# Unfreeze all of the layers in the base model
base.trainable = True

# Refreeze every layer except for the last 5
for layer in base.layers[:-5]:
  layer.trainable = False

we must compile the model again after fine tuning it.  if at all the model doesnt perform well, we can always revert back to the previous model

In [ ]:
# Recompile model with lower learning rate
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # 10x lower learning rate than default
              metrics=['accuracy'])

lets identify which models are trainable

In [ ]:
# What layers in the model are trainable?
for layer in model_0.layers:
  print(layer.name, layer.trainable)

input_layer True
data_augmentation True
efficientnetb0 True
global_average_pooling_layer True
output_layer True


lets work out model now with fit

In [ ]:
fine_tuned = model_0.fit(trainData,
                         epochs= 15,
                         validation_data= testData,
                         validation_steps= len(testData),
                         steps_per_epoch= len(trainData),
                         initial_epoch=history.epoch[-1])

Epoch 10/15


237/237 [==============================] - 105s 397ms/step - loss: 1.1367 - accuracy: 0.6936 - val_loss: 1.9240 - val_accuracy: 0.5183
Epoch 11/15
237/237 [==============================] - 91s 380ms/step - loss: 1.0299 - accuracy: 0.7216 - val_loss: 1.9206 - val_accuracy: 0.5199
Epoch 12/15
237/237 [==============================] - 92s 387ms/step - loss: 0.9493 - accuracy: 0.7419 - val_loss: 1.9426 - val_accuracy: 0.5204
Epoch 13/15
237/237 [==============================] - 89s 375ms/step - loss: 0.9129 - accuracy: 0.7535 - val_loss: 1.9392 - val_accuracy: 0.5238
Epoch 14/15
237/237 [==============================] - 89s 376ms/step - loss: 0.8663 - accuracy: 0.7650 - val_loss: 1.9471 - val_accuracy: 0.5233
Epoch 15/15
237/237 [==============================] - 92s 387ms/step - loss: 0.7961 - accuracy: 0.7827 - val_loss: 1.9638 - val_accuracy: 0.5212
